# Pandas и большие файлы

In [1]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [3]:
sales_db = pd.read_csv('sales_db.csv')

In [6]:
sales_db.groupby('user_id').agg({'cost':max})

,cost
user_id,
1,199
2,35
3,8
9,17


# Объединение датафреймов

### Данные со слайдов

In [7]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [8]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [9]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [22]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [23]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [24]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [25]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [26]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [27]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [10]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [11]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [14]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [15]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [18]:
merged = (earnings.merge(spending, on='id', how='outer')).merge(clients, on='id', how='outer')

In [19]:
merged

,id,debit,credit,name
0,51043,34500,22990,Благова Дарья
1,48329,12400,2500,Зарицкая Елизавета
2,74943,89044,69880,Слепова Елена
3,75029,5355,6000,Гордецкий Максим
4,43018,19800,29000,Марков Илья


In [20]:
merged.loc[merged.credit > merged.debit]

,id,debit,credit,name
3,75029,5355,6000,Гордецкий Максим
4,43018,19800,29000,Марков Илья


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [28]:
visits_grouped.join(purchases_pivot, how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [29]:
visits_grouped.join(purchases_pivot, how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


# Конкатенация таблиц

In [ ]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [ ]:
a

In [ ]:
b

In [ ]:
pd.concat([a, b])

In [ ]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

### Дубликаты при объединении таблиц

In [ ]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

In [ ]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

In [ ]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

In [ ]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

In [ ]:
ratings.merge(movies, how = 'left', on = 'movieId')

In [ ]:
ratings.merge(movies, how = 'right', on = 'movieId')

### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [45]:
def conc(arg):
    sources = []
    for source in arg:
        sources.append(source)
    return sources

In [46]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits.groupby('date').agg({'source' : conc, 'visits' : sum})

,source,visits
date,,
2019-11-01,"[organic, paid]",18777
2019-11-02,"[organic, paid]",22266
2019-11-03,[organic],19509


In [31]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [32]:
visits.merge(orders, on='date', how='outer')

,date,source,visits,orders
0,2019-11-01,organic,16825,198
1,2019-11-01,paid,1952,198
2,2019-11-02,organic,21890,225
3,2019-11-02,paid,376,225
4,2019-11-03,organic,19509,201


### Оптимизация хранения данных

In [47]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [48]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [49]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [50]:
len(ratings)

100004

In [51]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [52]:
len(movies)

9125

In [53]:
len(ratings) == len(joined)

True

In [54]:
joined.to_csv('joined_ratings.csv', index=False)

In [55]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [56]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [57]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

9066

### Какой жанр имеет самые высокие рейтинги?

In [75]:
import numpy as np

In [76]:
genres = ['Drama', 'Action', 'Thriller']

In [77]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [78]:
len(ratings)

100004

In [79]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [63]:
len(movies)

9125

In [80]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [65]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [66]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [94]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [95]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

CPU times: user 4min 55s, sys: 1.31 s, total: 4min 56s
Wall time: 4min 56s


In [98]:
joined.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres', 'Drama',
       'Action', 'Thriller', 'Adventure', 'Animation', '(no genres listed)',
       'Comedy', 'Romance', 'Film-Noir', 'Sci-Fi', 'Fantasy', 'War', 'Horror',
       'IMAX', 'Crime', 'Children', 'Western', 'Musical', 'Mystery',
       'Documentary'],
      dtype='object')

In [85]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [86]:
joined[genres] = joined.apply(genres_ratings_version_2, axis=1)
joined.head()

NameError: name 'rating' is not defined

In [99]:
joined.describe

<bound method NDFrame.describe of         userId  movieId  rating   timestamp  \
0            1       31     2.5  1260759144   
1            1     1029     3.0  1260759179   
2            1     1061     3.0  1260759182   
3            1     1129     2.0  1260759185   
4            1     1172     4.0  1260759205   
...        ...      ...     ...         ...   
99999      671     6268     2.5  1065579370   
100000     671     6269     4.0  1065149201   
100001     671     6365     4.0  1070940363   
100002     671     6385     2.5  1070979663   
100003     671     6565     3.5  1074784724   

                                                 title  \
0                               Dangerous Minds (1995)   
1                                         Dumbo (1941)   
2                                      Sleepers (1996)   
3                          Escape from New York (1981)   
4       Cinema Paradiso (Nuovo cinema Paradiso) (1989)   
...                                                ..

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

In [92]:
genres = list(set('|'.join(movies.genres.unique()).split('|')))

In [93]:
genres

['Adventure',
 'Animation',
 '(no genres listed)',
 'Comedy',
 'Romance',
 'Film-Noir',
 'Thriller',
 'Sci-Fi',
 'Fantasy',
 'War',
 'Horror',
 'IMAX',
 'Crime',
 'Children',
 'Western',
 'Musical',
 'Mystery',
 'Documentary',
 'Drama',
 'Action']

In [115]:
joined[genres].mean()

Adventure             3.520393
Animation             3.636062
(no genres listed)    3.777778
Comedy                3.446037
Romance               3.556165
Film-Noir             3.955702
Thriller              3.518502
Sci-Fi                3.460430
Fantasy               3.518063
War                   3.817214
Horror                3.315243
IMAX                  3.571134
Crime                 3.679639
Children              3.466187
Western               3.566423
Musical               3.598793
Mystery               3.679541
Documentary           3.813299
Drama                 3.681780
Action                3.445613
dtype: float64

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [116]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [117]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [118]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [119]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4
